#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation for Adult race.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

In [1]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd
import pickle

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

#### Load dataset and specify options


Huangrui's ADULT PREPROCESS

Only modify the sex and race to 1/0

With favorable one as 1. 

keep all other features

Race Is not dummy columned, thus we have fewer columns


# Huangrui defined Adult Dataset

In [2]:
default_mappings = {
    'label_maps': [{1.0: '>50K', 0.0: '<=50K'}],
    'protected_attribute_maps': [{1.0: 'Non-Black', 0.0: 'Black'},
                                 {1.0: 'Male', 0.0: 'Female'}]
}
def label_race(row):
   if row['race_Black'] == 1:
      return 0
   return 1
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
class AdultDataset(StandardDataset):
    """Adult Census Income Dataset.

    See :file:`aif360/data/raw/adult/README.md`.
    """

    def __init__(self, label_name='Y',
                 favorable_classes=[1],
                 protected_attribute_names=['sex_Female'],
                 privileged_classes=[[0]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[], features_to_drop=[],
                 na_values=['?'], custom_preprocessing=None,
                 metadata=default_mappings, path=None):
        #deal with the sex
        df = pd.read_csv(path,index_col=False,header = 0, sep = ',')
        # Here we drop the sex_Male column just to make sure the data is the same so that there is no 
        # error for our pretrained initial biased model
        # Considering our privileged group is "Male", here, privileged_class is 0
        df.drop(['sex_Male'],axis = 1,inplace = True)
        numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
        code_continuous(df,numericals, 5)
        super(AdultDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)


In [3]:
## import dataset
dataset_used = "adult" 

# Considering our privileged group is "Male", here, privileged_class is 0
privileged_groups = [{'sex_Female': 0}]
unprivileged_groups = [{'sex_Female': 1}]

# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

### Train classifier on original data

In [6]:
experiments_info = {}
budget = 0.01
for K in range(1, 6):
    dataset_orig_train= AdultDataset(path="./Huangrui/adult/adult_train{}.csv".format(K),protected_attribute_names=['sex_Female'],
                privileged_classes=[[0]])
    dataset_orig_test= AdultDataset(path="./Huangrui/adult/adult_test{}.csv".format(K),protected_attribute_names=['sex_Female'],
                privileged_classes=[[0]])
    #only use the budget% of the training data
    dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)

    # Lasso linear classifier and predictions
    X_train = dataset_orig_train.features
    y_train = dataset_orig_train.labels.ravel()
    lmod = pickle.load(open('experiments/adult'+str(K)+'_sex_bmodel.pkl','rb'))["clf"]
   
    y_train_pred = lmod.predict(X_train)

    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_train_pred.labels = y_train_pred.reshape(-1,1)
    sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
    dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)

    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = dataset_orig_test_pred.features
    y_test = dataset_orig_test_pred.labels
    dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)

    #### Best threshold for classification only (no fairness)
    num_thresh = 100
    ba_arr = np.zeros(num_thresh)
    class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
    for idx, class_thresh in enumerate(class_thresh_arr):
        
        fav_inds = dataset_orig_train_pred.scores > class_thresh
        dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
        dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label
        
        classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                                dataset_orig_train_pred, 
                                                unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        
        ba_arr[idx] = 0.5*(classified_metric_orig_train.true_positive_rate()\
                        +classified_metric_orig_train.true_negative_rate())

    best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
    best_class_thresh = class_thresh_arr[best_ind]

    # print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
    # print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)
    #### Estimate optimal parameters for the ROC method
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    low_class_thresh=0.01, high_class_thresh=0.99,
                                    num_class_thresh=100, num_ROC_margin=50,
                                    metric_name=metric_name,
                                    metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(dataset_orig_train, dataset_orig_train_pred)
    # print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    # print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # Metrics for the train set
    fav_inds = dataset_orig_train_pred.scores > best_class_thresh
    dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
    dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label

    # display(Markdown("#### train set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_train_bef = compute_metrics(dataset_orig_train, dataset_orig_train_pred, 
                    unprivileged_groups, privileged_groups)

    # Transform the validation set
    dataset_transf_train_pred = ROC.predict(dataset_orig_train_pred)

    # display(Markdown("#### train set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_train_aft = compute_metrics(dataset_orig_train, dataset_transf_train_pred, 
                    unprivileged_groups, privileged_groups)
    # Testing: Check if the metric optimized has not become worse
    assert np.abs(metric_train_aft[metric_name]) <= np.abs(metric_train_bef[metric_name])

    # Metrics for the test set
    fav_inds = dataset_orig_test_pred.scores > best_class_thresh
    dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
    dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                    unprivileged_groups, privileged_groups)
    # Metrics for the transformed test set
    dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                    unprivileged_groups, privileged_groups)

    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}

C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.8444
Statistical parity difference = -0.1816
Disparate impact = 0.4587
Average odds difference = -0.0461
Equal opportunity difference = 0.0182
Theil index = 0.0759
Balanced accuracy = 0.8444
Statistical parity difference = -0.1816
Disparate impact = 0.4587
Average odds difference = -0.0461
Equal opportunity difference = 0.0182
Theil index = 0.0759
Balanced accuracy = 0.7731
Statistical parity difference = -0.2290
Disparate impact = 0.3100
Average odds difference = -0.1378
Equal opportunity difference = -0.1410
Theil index = 0.1147
Balanced accuracy = 0.7731
Statistical parity difference = -0.2290
Disparate impact = 0.3100
Average odds difference = -0.1378
Equal opportunity difference = -0.1410
Theil index = 0.1147
K = 1, budget = 0.01
The Error for the test dataset is 0.1743
The Equal opportunity difference for the test dataset is -0.141


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.7977
Statistical parity difference = -0.1929
Disparate impact = 0.5058
Average odds difference = -0.1060
Equal opportunity difference = -0.1043
Theil index = 0.0930
Balanced accuracy = 0.7943
Statistical parity difference = -0.1650
Disparate impact = 0.6660
Average odds difference = -0.0439
Equal opportunity difference = 0.0087
Theil index = 0.0785
Balanced accuracy = 0.7851
Statistical parity difference = -0.2615
Disparate impact = 0.3316
Average odds difference = -0.1386
Equal opportunity difference = -0.1143
Theil index = 0.1016
Balanced accuracy = 0.7785
Statistical parity difference = -0.1771
Disparate impact = 0.6418
Average odds difference = -0.0242
Equal opportunity difference = 0.0391
Theil index = 0.0861
K = 2, budget = 0.01
The Error for the test dataset is 0.2662
The Equal opportunity difference for the test dataset is 0.03913


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.7923
Statistical parity difference = -0.3110
Disparate impact = 0.4457
Average odds difference = -0.1917
Equal opportunity difference = -0.1348
Theil index = 0.0756
Balanced accuracy = 0.7923
Statistical parity difference = -0.1280
Disparate impact = 0.6613
Average odds difference = -0.0181
Equal opportunity difference = 0.0174
Theil index = 0.0945
Balanced accuracy = 0.7976
Statistical parity difference = -0.3402
Disparate impact = 0.3841
Average odds difference = -0.1657
Equal opportunity difference = -0.0762
Theil index = 0.0787
Balanced accuracy = 0.7889
Statistical parity difference = -0.1756
Disparate impact = 0.5448
Average odds difference = -0.0108
Equal opportunity difference = 0.0583
Theil index = 0.0989
K = 3, budget = 0.01
The Error for the test dataset is 0.1995
The Equal opportunity difference for the test dataset is 0.05828


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.7977
Statistical parity difference = -0.1929
Disparate impact = 0.5058
Average odds difference = -0.1060
Equal opportunity difference = -0.1043
Theil index = 0.0930
Balanced accuracy = 0.7958
Statistical parity difference = -0.1271
Disparate impact = 0.6743
Average odds difference = -0.0257
Equal opportunity difference = -0.0043
Theil index = 0.0914
Balanced accuracy = 0.7871
Statistical parity difference = -0.2603
Disparate impact = 0.3104
Average odds difference = -0.1540
Equal opportunity difference = -0.1511
Theil index = 0.1040
Balanced accuracy = 0.7887
Statistical parity difference = -0.1924
Disparate impact = 0.4965
Average odds difference = -0.0473
Equal opportunity difference = 0.0053
Theil index = 0.1004
K = 4, budget = 0.01
The Error for the test dataset is 0.1947
The Equal opportunity difference for the test dataset is 0.005324


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.7927
Statistical parity difference = -0.2895
Disparate impact = 0.4211
Average odds difference = -0.2012
Equal opportunity difference = -0.1913
Theil index = 0.0845
Balanced accuracy = 0.7877
Statistical parity difference = -0.1454
Disparate impact = 0.6441
Average odds difference = -0.0384
Equal opportunity difference = -0.0043
Theil index = 0.0934
Balanced accuracy = 0.8064
Statistical parity difference = -0.3251
Disparate impact = 0.3386
Average odds difference = -0.1799
Equal opportunity difference = -0.1366
Theil index = 0.0843
Balanced accuracy = 0.8029
Statistical parity difference = -0.2202
Disparate impact = 0.4692
Average odds difference = -0.0749
Equal opportunity difference = -0.0371
Theil index = 0.0916
K = 5, budget = 0.01
The Error for the test dataset is 0.1959
The Equal opportunity difference for the test dataset is -0.03707


In [7]:
experiments_info

{'K = 1, budget = 0.01': {'Error': 0.1743042826195634,
  'Equal opportunity difference': -0.14095107765993842},
 'K = 2, budget = 0.01': {'Error': 0.26616666666666666,
  'Equal opportunity difference': 0.039134445890512515},
 'K = 3, budget = 0.01': {'Error': 0.1995,
  'Equal opportunity difference': 0.058281798297376786},
 'K = 4, budget = 0.01': {'Error': 0.19466666666666665,
  'Equal opportunity difference': 0.00532350532350534},
 'K = 5, budget = 0.01': {'Error': 0.19586597766294384,
  'Equal opportunity difference': -0.03706631765293311}}